In [0]:
from pyspark.sql.functions import row_number, desc, md5, col
from pyspark.sql.window import Window

In [0]:
major_incidents_df = spark.read \
    .table("mta_bronze.major_incident")

In [0]:
inc_category_df = major_incidents_df \
    .select("inc_category").distinct() \
    .orderBy(desc("inc_category")) \
    .withColumn("ict_nk", row_number().over(Window.orderBy(desc("inc_category")))) \
    .withColumn("ict_sk", md5("inc_category"))

In [0]:
inc_category_final_df = inc_category_df.select(
    col("ict_sk"),
    col("ict_nk"),
    col("inc_category").alias("ict_category")
)

In [0]:
inc_category_final_df.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("mta_silver.dim_inc_category")

In [0]:
%sql
SELECT * FROM mta_silver.dim_inc_category;

In [0]:
dbutils.notebook.exit("Success")